In [3]:
!pip install pyTigerGraphBeta
!pip install flat-table

import pyTigerGraphBeta as tg
import pandas as pd
import flat_table

In [4]:
conn = tg.TigerGraphConnection(host="https://aml-sim-sagemaker.i.tgcloud.io", username="tigergraph", password="tigergraph", graphname = "AMLSim")
conn.apiToken = conn.getToken("0dqle85rg436lg25qabtki0lqenunfvj")

In [5]:
tx_hop = conn.runInstalledQuery("txMultiHopLimit", {}, timeout="10000000000000000", sizeLimit="1500000000")

In [6]:
df_tx_hop = pd.DataFrame(tx_hop[0]["@@txRecords"])
df_tx_hop = flat_table.normalize(df_tx_hop)
df_tx_hop.head()

,index,tx_amount,tx_fraud,s_pagerank,s_label,s_min_send_tx,s_min_receieve_tx,s_max_send_tx,s_max_recieve_tx,s_avg_send_tx,...,r_label,r_min_send_tx,r_min_receieve_tx,r_max_send_tx,r_max_recieve_tx,r_avg_send_tx,r_avg_recieve_tx,r_cnt_recieve_tx,r_cnt_send_tx,r_timestamp
0,0,0.0,False,4.37799,58720367,0.0,3.73,16147263.00,21474836.47,3.424254e+06,...,17825845,564.67,0.0,564.67,2823050.75,5.646700e+02,67687.90131,61,177,0
1,1,0.0,False,0.94411,10485846,0.0,12.55,14930847.00,11150681.00,1.850771e+06,...,52428894,3461.37,0.0,21474836.47,8101879.00,4.897379e+06,65856.86086,187,30,0
2,2,0.0,False,0.62607,41943056,0.0,14.54,13832755.00,21474836.47,4.111439e+06,...,10485767,1200.91,0.0,21474836.47,13832713.00,5.057516e+06,59041.37387,297,16,0
3,3,0.0,False,0.58950,13631539,0.0,166.02,21474836.47,30921.03,3.618203e+06,...,59768883,118.33,0.0,118.33,21474836.47,1.183300e+02,140310.31232,521,720,0
4,4,0.0,False,0.57978,35,0.0,105.06,10701302.00,21474836.47,1.659735e+06,...,37748786,11.85,0.0,493.54,21474836.47,4.908787e+02,93889.93339,655,181,1


In [10]:
df_tx_hop.to_csv(r'./tmp/20210412AMLsim.csv', index = False, header=True)

# SageMaker Model Generation 


In [11]:
import sagemaker
from sagemaker import get_execution_role

In [12]:
sagemaker_session = sagemaker.Session()

In [13]:
# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [14]:
role

'arn:aws:iam::816500858139:role/service-role/AmazonSageMaker-ExecutionRole-20210412T170094'

## Upload the data for training 


In [15]:
train_input = sagemaker_session.upload_data("tmp")

In [16]:
train_input

's3://sagemaker-us-east-2-816500858139/data'

# Create SageMaker Scikit Estimator

In [24]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'fraud-prediction.py'

sklearn = SKLearn(
    entry_point=script_path,
    instance_type="ml.m4.xlarge",
    framework_version="0.20.0",
    py_version="py3",
    role=role,
    sagemaker_session=sagemaker_session)

# Train SKLearn Estimator on Startup data 


In [25]:
sklearn.fit({'train': train_input})

2021-04-13 00:48:50 Starting - Starting the training job...
2021-04-13 00:49:13 Starting - Launching requested ML instancesProfilerReport-1618274929: InProgress
......
2021-04-13 00:50:13 Starting - Preparing the instances for training.........
2021-04-13 00:51:34 Downloading - Downloading input data...
2021-04-13 00:52:21 Training - Training image download completed. Training in progress..2021-04-13 00:52:21,906 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-04-13 00:52:21,909 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-13 00:52:21,921 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-04-13 00:52:22,313 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-13 00:52:25,349 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-13 00:52:25,365 sagemaker-training-toolkit INFO     No GPUs detecte

# Deploy the model 

In [26]:
deployment = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

---------------!

In [27]:
deployment.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-scikit-learn-2021-04-13-00-53-12-835'